In [5]:
import sys
import os
sys.path.append('..')
import pickle
import numpy as np
from src.DataLoader import LoadPickleData
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import classification_report, confusion_matrix
print(os.getcwd())
def get_top_k_metric(k, y_predict, test_set_y):
        precision, recall = 0, 0
        tp, fp, fn = 0, 0, 0
        pt_pair = []
        for x, y in zip(y_predict, test_set_y):
            pt_pair.append([x, y])
            
        pt_pair.sort(key=lambda x: x[0], reverse=True)
        
        n_rel = sum((true_r == 1) for (_, true_r) in pt_pair)

        # Number of recommended items in top k
        n_rec_k = sum((est == 1) for (est, _) in pt_pair[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r == 1) and (est == 1))
                              for (est, true_r) in pt_pair[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precision = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recall = n_rel_and_rec_k / n_rel if n_rel != 0 else 0
        
        return n_rel_and_rec_k, precision, recall

/home/swj/VD/src


In [7]:
#In domain
repre = LoadPickleData('/home/swj/VD/Representation/PLSTM/FFmpeg_Vul.pkl')
non_repre = LoadPickleData('/home/swj/VD/Representation/PLSTM/FFmpeg_NonVul.pkl')

n, m = len(repre), len(non_repre)
n_features = 128
train_set_x = []
for x in repre:
    train_set_x.append(x)
for x in non_repre:
    train_set_x.append(x)
X = np.asarray(train_set_x)
y = np.asarray([1] * n + [0] * m)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=321)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
n_vul = sum((y == 1) for y in y_test)
print(n_vul)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
clf = RandomForestClassifier(bootstrap=True, class_weight='balanced', #class_weight={0:1, 1:4},
            criterion='entropy', max_depth=40, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=3,
            min_samples_split=4, min_weight_fraction_leaf=0.0,
            n_estimators=8000, oob_score=False, random_state=None,
            verbose=1, warm_start=False, n_jobs=-1)
clf = clf.fit(X_train, y_train)
# print("feature importance:")
# print(clf.feature_importances_)
# print ("\n")

print ("\r\n")
    
    #evaluate the model on the test set
print("predicting on the test set")
y_predict = clf.predict(X_test)
    
y_predict_proba = clf.predict_proba(X_test)
# print(y_test)
# print(y_predict, y_predict_proba)
k_list = [10, 20, 30, 40, 50, 100, 150, 200]
for k in k_list:
    n_vul, precision, recall = get_top_k_metric(k, y_predict, y_test)
    print('nums of vul, precision and recall at {0} is {1}, {2} and {3}'.format(k, n_vul, precision, recall))

    # Accuracy
accuracy = np.mean(y_test==y_predict)*100
print ("accuracy = " +  str(accuracy))
        
target_names = ["Non-defective","Defective"] #non-buggy->0, buggy->1
print (confusion_matrix(y_test, y_predict, labels=[0,1]))   
print ("\r\n")
print ("\r\n")
print (classification_report(y_test, y_predict, target_names=target_names))

81
(4060, 128) (1741, 128) (4060,) (1741,)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 6034 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done 7184 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 8000 out of 8000 | elapsed:    6.5s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jo



predicting on the test set


[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed:    1.9s
[Parallel(n_jobs=8)]: Done 4034 tasks      | elapsed:    2.7s
[Parallel(n_jobs=8)]: Done 4984 tasks      | elapsed:    3.5s
[Parallel(n_jobs=8)]: Done 6034 tasks      | elapsed:    4.4s
[Parallel(n_jobs=8)]: Done 7184 tasks      | elapsed:    5.4s
[Parallel(n_jobs=8)]: Done 8000 out of 8000 | elapsed:    6.1s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done 1784 tasks      | e

nums of vul, precision and recall at 10 is 8, 0.8 and 0.09876543209876543
nums of vul, precision and recall at 20 is 18, 0.9 and 0.2222222222222222
nums of vul, precision and recall at 30 is 27, 0.9 and 0.3333333333333333
nums of vul, precision and recall at 40 is 37, 0.925 and 0.4567901234567901
nums of vul, precision and recall at 50 is 46, 0.92 and 0.5679012345679012
nums of vul, precision and recall at 100 is 72, 0.935064935064935 and 0.8888888888888888
nums of vul, precision and recall at 150 is 72, 0.935064935064935 and 0.8888888888888888
nums of vul, precision and recall at 200 is 72, 0.935064935064935 and 0.8888888888888888
accuracy = 99.19586444572084
[[1655    5]
 [   9   72]]




               precision    recall  f1-score   support

Non-defective       0.99      1.00      1.00      1660
    Defective       0.94      0.89      0.91        81

     accuracy                           0.99      1741
    macro avg       0.96      0.94      0.95      1741
 weighted avg       0.9

[Parallel(n_jobs=8)]: Done 8000 out of 8000 | elapsed:    6.9s finished


In [ ]:
#cross domain
train_set_vul, train_set_nonvul = [], []
train_set_vul_path = 'Representation/Train_set1/Vul/'
train_set_nonvul_path = 'Representation/Train_set1/NonVul/'

test_set_vul, test_set_nonvul = [], []
test_set_vul_path = 'Representation/Test_set1/Vul/'
test_set_nonvul_path = 'Representation/Test_set1/NonVul/'

files = os.listdir(train_set_vul_path)
for file in files:
    _, ext = os.path.splitext(file)
    _, filename = os.path.split(file)
    if ext == '.pkl':
        full_path = ''
        train_set_vul.append(LoadPickleData(train_set_vul_path + filename))
        
files = os.listdir(train_set_nonvul_path)
for file in files:
    _, ext = os.path.splitext(file)
    _, filename = os.path.split(file)
    if ext == '.pkl':
        full_path = ''
        train_set_nonvul.append(LoadPickleData(train_set_nonvul_path + filename))
        
files = os.listdir(test_set_vul_path)
for file in files:
    _, ext = os.path.splitext(file)
    _, filename = os.path.split(file)
    if ext == '.pkl':
        full_path = ''
        test_set_vul.append(LoadPickleData(test_set_vul_path + filename))
        
files = os.listdir(test_set_nonvul_path)
for file in files:
    _, ext = os.path.splitext(file)
    _, filename = os.path.split(file)
    if ext == '.pkl':
        full_path = ''
        test_set_nonvul.append(LoadPickleData(test_set_nonvul_path + filename))
        
train_set_x = []
num_vul, num_nonvul = 0, 0
for vul in train_set_vul:
    for x in vul:
        train_set_x.append(x)
        num_vul += 1
for nonvul in train_set_nonvul:
    for x in nonvul:
        train_set_x.append(x)
        num_nonvul += 1
X_train = np.asarray(train_set_x)
y_train = np.asarray([1] * num_vul + [0] * num_nonvul)
print('train_set:')
print(X_train.shape, y_train.shape)

test_set_x = []
num_vul, num_nonvul = 0, 0
for vul in test_set_vul:
    for x in vul:
        test_set_x.append(x)
        num_vul += 1
for nonvul in test_set_nonvul:
    for x in nonvul:
        test_set_x.append(x)
        num_nonvul += 1
X_test = np.asarray(test_set_x)
y_test = np.asarray([1] * num_vul + [0] * num_nonvul)
print('test_set:')
print(X_test.shape, y_test.shape)


n_vul = sum((y == 1) for y in y_test)
print(n_vul)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
clf = RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='entropy', max_depth=30, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=3,
            min_samples_split=4, min_weight_fraction_leaf=0.0,
            n_estimators=8000, oob_score=False, random_state=None,
            verbose=1, warm_start=False, n_jobs=-1)
clf = clf.fit(X_train, y_train)

print ("\r\n")
    
#evaluate the model on the test set
print("predicting on the test set")
y_predict = clf.predict(X_test)
    
y_predict_proba = clf.predict_proba(X_test)
k_list = [10, 20, 30, 40, 50, 100, 150, 200]
for k in k_list:
    n_vul, precision, recall = get_top_k_metric(k, y_predict, y_test)
    print('nums of vul, precision and recall at {0} is {1}, {2} and {3}'.format(k, n_vul, precision, recall))

# Accuracy
accuracy = np.mean(y_test==y_predict)*100
print ("accuracy = " +  str(accuracy))
        
target_names = ["Non-defective","Defective"] #non-buggy->0, buggy->1
print (confusion_matrix(y_test, y_predict, labels=[0,1]))   
print ("\r\n")
print ("\r\n")
print (classification_report(y_test, y_predict, target_names=target_names))